# Optimization: The Data-Driven Diet



The "diet problem" is a classic optimization problem in operations research. Back in the 1930s and 1940s, the United States Army wanted to know what to feed its soldiers in order to satisfy healthy nutritional requirements at minimum cost. This research became one of the first large-scale optimization problems to be studied in practice.

The diet problem highlights the key aspects of optimization: reducing waste, minimizing cost, and meeting essential constraints. In this analysis, we will revisit the diet problem by applying an optimization model to a modern nutritional dataset. Our goal will be to find the cheapest diet possible that satisfies the maximum and minimum daily nutrional constraints, using the food items that are listed in the dataset. 

## Exploring the Data

We begin the analysis with an initial exploration of the data. 

In [7]:
import pandas as pd
import pulp 

file_path = r"E:\ISYE 6501\Week 11\HW11\data 15.2\diet.xls"

# Read the Excel file
data = pd.read_excel(file_path)

display(data)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0


Upon inspecting our dataset, we can see that the dataset contains 66 rows and 14 columns. Rows 1-63 contains the various food options that are available and the price/serving/nutritional information. Row 64 is a blank space and rows 65-66 are the minimum/maximum daily intake constraints for each nutrient. Before moving forward with the analysis, we will separate the food data and the nutrient constraints. 

In [11]:
# Select the first 63 rows of data, which contain the food items and their attributes 
food_data = data[:64]

# Isolate rows starting from row 65 and onward, and all columns starting from column 3
nutrient_constraints = data.iloc[65:, 3:].reset_index(drop=True)
display(food_data)
display(nutrient_constraints)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6


,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
1,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


Now, we will extract each column that corresponds to a specific nutrient:

In [277]:
# Extract costs and nutrient values for each food from 'food_data` 

calories = food_data['Calories'].tolist()
cholesterol = food_data['Cholesterol mg'].tolist()
fat = food_data['Total_Fat g'].tolist()
sodium = food_data['Sodium mg'].tolist()
carbs = food_data['Carbohydrates g'].tolist()
fiber = food_data['Dietary_Fiber g'].tolist()
protein = food_data['Protein g'].tolist()
vit_a = food_data['Vit_A IU'].tolist()
vit_c = food_data['Vit_C IU'].tolist()
calcium = food_data['Calcium mg'].tolist()
iron = food_data['Iron mg'].tolist()

## Create the LP Object

The data has been organized, and the next step is to set up a linear programming (LP) problem in the Python library PuLP. PuLP is a convenient software that allows us to set up optimization models, define constraints, and choose solvers to find optimal solutions. 

In [280]:
# PuLP Problem
prob = pulp.LpProblem("Diet_Optimization", sense=pulp.LpMinimize)

# Decision Variables
food_vars = [pulp.LpVariable(f"x_{food_data['Foods'][i]}", lowBound=0, cat='Continuous') for i in range(len(food_data))]

In this code, we have set up the LP Object and the decision variables for each food item has been designated by the optimization model. Each variable represents a decision we must make about how much of each food should we include in order to meet the daily nutrient constraints. 

## Setting Up the Objective Function

In [283]:
# Objective function: Minimize total cost
food_costs = food_data['Price/ Serving'].tolist()

# Add or assign objective functino to the LP problem
prob += pulp.lpSum([food_costs [i] * food_vars[i] for i in range(len(food_vars))]), "Total Cost"

The objective function represents the quantity to be minimized or maximized in the optimization process. Each decision variable `food_vars[i]` is multiplied by the price per serving `food_costs[i]`, giving us the cost of that particular food item. We use a loop to make sure that every food item is included and added to the objective function. This will give us the final "Total Cost" of all the food items. 

## Adding Constraints (Min and Max Nutrients)

In [286]:
# Add Calories Constraint
prob += pulp.lpSum([calories[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 0], "Min_Calories"
prob += pulp.lpSum([calories[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 0], "Max_Calories"

# Add Cholesterol mg Constraint
prob += pulp.lpSum([cholesterol[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 1], "Min_Cholesterol"
prob += pulp.lpSum([cholesterol[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 1], "Max_Cholesterol"

# Add Total_Fat g Constraint
prob += pulp.lpSum([fat[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 2], "Min_Fat"
prob += pulp.lpSum([fat[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 2], "Max_Fat"

# Add Sodium mg Constraint
prob += pulp.lpSum([sodium[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 3], "Min_Sodium"
prob += pulp.lpSum([sodium[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 3], "Max_Sodium"

# Add Carbohydrates g Constraint
prob += pulp.lpSum([carbs[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 4], "Min_Carbs"
prob += pulp.lpSum([carbs[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 4], "Max_Carbs"

# Add Dietary_Fiber g Constraint
prob += pulp.lpSum([fiber[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 5], "Min_Fiber"
prob += pulp.lpSum([fiber[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 5], "Max_Fiber"

# Add Protein g Constraint
prob += pulp.lpSum([protein[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 6], "Min_Protein"
prob += pulp.lpSum([protein[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 6], "Max_Protein"

# Add Vit_A IU Constraint
prob += pulp.lpSum([vit_a[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 7], "Min_VitA"
prob += pulp.lpSum([vit_a[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 7], "Max_VitA"

# Add Vit_C IU Constraint
prob += pulp.lpSum([vit_c[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 8], "Min_VitC"
prob += pulp.lpSum([vit_c[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 8], "Max_VitC"

# Add Calcium mg Constraint
prob += pulp.lpSum([calcium[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 9], "Min_Calcium"
prob += pulp.lpSum([calcium[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 9], "Max_Calcium"

# Add Iron mg Constraint
prob += pulp.lpSum([iron[i] * food_vars[i] for i in range(len(food_vars))]) >= nutrient_constraints.iloc[0, 10], "Min_Iron"
prob += pulp.lpSum([iron[i] * food_vars[i] for i in range(len(food_vars))]) <= nutrient_constraints.iloc[1, 10], "Max_Iron"


This is the code for the min/max nutrient constraints. Adding these constraints will make sure that our diet will meet the minimum and maximum nutrient requirements. For each nutrient, we once again rely on loops to get the nutrients from every food item in the data. 

## Solve and Retrieve the Outcome

In [289]:
# Solve the problem 
prob.solve()

# Print the results
print("Status:", pulp. LpStatus [prob.status])
for v in food_vars:
    print(f"Optimal Quantity of {v.name}:", v. varValue) 
print("Minimum Cost:", pulp.value(prob.objective))

Status: Optimal
Optimal Quantity of x_Frozen_Broccoli: 0.25960653
Optimal Quantity of x_Carrots,Raw: 0.0
Optimal Quantity of x_Celery,_Raw: 52.64371
Optimal Quantity of x_Frozen_Corn: 0.0
Optimal Quantity of x_Lettuce,Iceberg,Raw: 63.988506
Optimal Quantity of x_Peppers,_Sweet,_Raw: 0.0
Optimal Quantity of x_Potatoes,_Baked: 0.0
Optimal Quantity of x_Tofu: 0.0
Optimal Quantity of x_Roasted_Chicken: 0.0
Optimal Quantity of x_Spaghetti_W__Sauce: 0.0
Optimal Quantity of x_Tomato,Red,Ripe,Raw: 0.0
Optimal Quantity of x_Apple,Raw,W_Skin: 0.0
Optimal Quantity of x_Banana: 0.0
Optimal Quantity of x_Grapes: 0.0
Optimal Quantity of x_Kiwifruit,Raw,Fresh: 0.0
Optimal Quantity of x_Oranges: 2.2929389
Optimal Quantity of x_Bagels: 0.0
Optimal Quantity of x_Wheat_Bread: 0.0
Optimal Quantity of x_White_Bread: 0.0
Optimal Quantity of x_Oatmeal_Cookies: 0.0
Optimal Quantity of x_Apple_Pie: 0.0
Optimal Quantity of x_Chocolate_Chip_Cookies: 0.0
Optimal Quantity of x_Butter,Regular: 0.0
Optimal Quantity 

## Results

**Optimal Food Choices:**

Air-popped popcorn: 13.87 servings

Poached eggs: 0.14 servings

Oranges: 2.29 servings

Raw iceberg lettuce: 63.99 servings

Raw celery: 52.64 servings

Frozen broccoli: 0.26 servings


**Minimum Cost:** The cheapest combination of foods costs approximately $4.34


This is the cheapest combination of foods produced by the optimization model that meets all the nutritional requirements. The main bulk of the meal consists of popcorn, lettuce, and celery. Although this is in theory the cheapest possible meal, it is not a very realistic diet. We can't expect expect a soldier to eat over 60 servings of raw lettuce and 50 servings of raw celery with some popcorn. In the next question, we will add some additional constraints to modify this diet. 

## Modifying the Diet

In this part, we will add to our model the some new constraints:

a. If a food is selected, then a minimum of 1/10 serving must be chosen. 

In [294]:
food_selected = [pulp.LpVariable(f"y_{i}", cat='Binary') for i in range(len(food_data))]

# Minimum serving size if a food is selected
for i in range(len(food_data)):
    prob += food_vars[i] >= 0.1 * food_selected[i], f"Min_Serving_{i}"
    
# Link binary and continuous variable so that if `y_i` = 0, `x_i` must also = 0
for i in range(len(food_data)):
    prob += food_vars[i] <= food_selected[i] * 100, f"Link_x_y_{i}" 

The code for this problem relies on binary variables to determine whether a food item is selected or not. The binary variable is then linked to a constraint making sure a food is selected, at least 1/10 will be included. 

b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

In [297]:
# Celery/Broccoli Constraint
prob += food_selected[2] + food_selected[0] <= 1, "Celery_Broccoli_Constraint"

Here, `food_selected[2]` is the binary variable for celery and `food_selected[0]` is the binary variable for broccoli. Making sure that the sum of these binary variables is less than equal ensures that at most one of them will be selected. 

c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be
selected.

In [300]:
# Protein Constraints
proteins = [
    "Roasted Chicken",
    "Poached Eggs",
    "Scrambled Eggs",
    "Bologna, Turkey",
    "Frankfurter, Beef",
    "Ham, Sliced, Extralean",
    "Kielbasa, Pork",
    "Pork",
    "Sardines in Oil",
    "White Tuna in Water"
]

protein_indices = []

for i in range(len(food_data['Foods'])):
    if food_data['Foods'][i] in proteins:
        protein_indices.append(i)

print(protein_indices) 

prob += pulp.lpSum([food_selected[i] for i in protein_indices]) >= 3, "Protein_Variety_Constraint"

[8, 27, 28, 30, 49, 50, 51]


For this problem, the following were selected as the main protein group: 

Roasted Chicken,
Poached Eggs,
Scrambled Eggs,
Bologna Turkey,
Frankfurter Beef
Ham Sliced Extralean,
Kielbasa Pork,
Pork,
Sardines in Oil,
White Tuna in Water

We first extract the indices of these protein food items and loop through them using the PuLP software. We designate that the sum of these protein binary variables will be equal to or greater than 3. This will ensure that at least 3 of these food items will be selected. 




## Solve the New Optimization Model

In [303]:
prob.solve()

print("Status:", pulp.LpStatus[prob.status])
for v in food_vars:
    print(f"Optimal Quantity of {v.name}:", v.varValue)
print("Minimum Cost:", pulp.value(prob.objective))

Status: Optimal
Optimal Quantity of x_Frozen_Broccoli: 0.0
Optimal Quantity of x_Carrots,Raw: 0.0
Optimal Quantity of x_Celery,_Raw: 42.032842
Optimal Quantity of x_Frozen_Corn: 0.0
Optimal Quantity of x_Lettuce,Iceberg,Raw: 83.647733
Optimal Quantity of x_Peppers,_Sweet,_Raw: 0.0
Optimal Quantity of x_Potatoes,_Baked: 0.0
Optimal Quantity of x_Tofu: 0.0
Optimal Quantity of x_Roasted_Chicken: 0.0
Optimal Quantity of x_Spaghetti_W__Sauce: 0.0
Optimal Quantity of x_Tomato,Red,Ripe,Raw: 0.0
Optimal Quantity of x_Apple,Raw,W_Skin: 0.0
Optimal Quantity of x_Banana: 0.0
Optimal Quantity of x_Grapes: 0.0
Optimal Quantity of x_Kiwifruit,Raw,Fresh: 0.0
Optimal Quantity of x_Oranges: 3.0746034
Optimal Quantity of x_Bagels: 0.0
Optimal Quantity of x_Wheat_Bread: 0.0
Optimal Quantity of x_White_Bread: 0.0
Optimal Quantity of x_Oatmeal_Cookies: 0.0
Optimal Quantity of x_Apple_Pie: 0.0
Optimal Quantity of x_Chocolate_Chip_Cookies: 0.0
Optimal Quantity of x_Butter,Regular: 0.0
Optimal Quantity of x_C

**Optimal Food Choices**

Raw Celery: 42.03 grams

Raw Iceburg Lettuce: 83.65 grams

Oranges: 3.07 grams

Poached Eggs: 0.1 servings

Scrambled Eggs: 0.1 servings

Frankfurter Beef: 0.1 servings

Peanut Butter: 1.90 grams

Air Popped Popcorn: 13.25 grams


**Minimum Cost**: The cheapest combination of foods costs approximately $4.52

## Conclusion

This is the new optimal meal that includes all of the new constraints. It is a slightly more feasible meal in terms of diversity. Peanut butter, poached eggs, scrambled eggs, and frakfurter beef was added to the meal. The model has chosen to keep the lettuce and take out broccoli. It seems that this new diet plan is a bit more expensive than that of our previous model, but not by a significant amount (about 18 cents). The meal does meet the nutritional requirements but it still doesn't look like a realistic diet. The meal relies heavily on raw celery and raw iceburg lettuce to meet the constraints, and it feels like everything else is there for garnish. If desired, we can further this analysis by increasing the amount of other foods to make it a more balanced meal. We can do this by requiring additional constraints added to the model. 

Mathematics and computer software may give us a perfect meal in theory, but it is not a meal that we could use in a realistic diet plan. The model may provide a mathematically ideal answer, but models must often be adjusted for practicality and context. This is an important lesson in optimization modeling. In order to fully utilize optimization models to real life situations, human judgment and wisdom is required. Like any aspect of technology, optimization is a tool and the user is the one who decides how to best apply the results. 